## Importing Libraries

In [0]:
import numpy as np
import pandas as pd

## Extracting Global_active_power Values

In [0]:
#reading global active power values in list
input_list=[]
fp=open('/content/drive/My Drive/input.txt')
for i in fp:
    ip=i.split(';')
    input_list.append(ip[2])

In [0]:
print(len(input_list))
inp=np.array(input_list)
inputData=inp[1:]
print(len(inputData))

2075260
2075259


## TrainData Preprocessing

In [0]:
processedlist = inputData[inputData != '?']

In [0]:
print(len(processedlist))

2049280


In [0]:
dataframe=pd.DataFrame(processedlist)
dataset = dataframe.values
dataset = dataset.astype('float32')

In [0]:
print(dataset)

[[4.216]
 [5.36 ]
 [5.374]
 ...
 [0.938]
 [0.934]
 [0.932]]


In [0]:
print(dataset.shape)

(2049280, 1)


In [0]:
def divide_chunks(l,step): 
  for i in range(0, len(l)-step):  
    yield l[i:i+step]
step=60
x = list(divide_chunks(dataset))
x=np.array(x)
x.shape=(2049220,60)
y_train=x[:,59:]
y_train=y_train[1:]
y_train=np.append(y_train,1.0)
l=len(y_train)
print(y_train.size)

2049220


In [0]:
print(x.shape)
print(y_train.shape)

(2049220, 60)
(2049220,)


In [0]:
print(y_train)
labels=y_train

[3.4519999  4.86999989 4.86800003 ... 0.93800002 0.93400002 1.        ]


## Splitting DataFrame

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,labels,test_size=0.2)

## Training Model on MLP

### Importing Libraries

In [0]:
import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers

### Training MLP Model

In [0]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=60))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [0]:
model.fit(X_train,y_train, epochs=25, verbose=1)

Epoch 1/25
51231/51231 [==============================] - 71s 1ms/step - loss: 0.0724
Epoch 2/25
51231/51231 [==============================] - 71s 1ms/step - loss: 0.0678
Epoch 3/25
51231/51231 [==============================] - 71s 1ms/step - loss: 0.0667
Epoch 4/25
51231/51231 [==============================] - 72s 1ms/step - loss: 0.0660
Epoch 5/25
51231/51231 [==============================] - 71s 1ms/step - loss: 0.0656
Epoch 6/25
51231/51231 [==============================] - 71s 1ms/step - loss: 0.0652
Epoch 7/25
51231/51231 [==============================] - 71s 1ms/step - loss: 0.0647
Epoch 8/25
51231/51231 [==============================] - 72s 1ms/step - loss: 0.0644
Epoch 9/25
51231/51231 [==============================] - 72s 1ms/step - loss: 0.0640
Epoch 10/25
51231/51231 [==============================] - 71s 1ms/step - loss: 0.0638
Epoch 11/25
51231/51231 [==============================] - 71s 1ms/step - loss: 0.0636
Epoch 12/25
51231/51231 [===========================

In [0]:
print(X_test.shape)

(409844, 60)


### Accuracy On Test Data

In [0]:
pred = model.predict(X_test, verbose=1)
# print(yhat)
from sklearn.metrics import r2_score
a = r2_score(pred,y_test)
print('r2 score is:', a*100)

12808/12808 [==============================] - 12s 952us/step
r2 score is: 93.6303259798334


## Predicting Missing Values 

In [0]:
print(inputData)

['4.216' '5.360' '5.374' ... '0.938' '0.934' '0.932']


In [0]:
predlist=[]
for i in range(0,len(inputData)-60):
  if(inputData[i+60]=='?'):
    X_test=inputData[i:i+60]
    X_test=np.array(X_test)
    X_test= X_test.astype('float32')
    X_test.shape=(1,60)
    pred=model.predict(X_test, verbose=1)
    pred.shape=(1,)
    print(pred[0])
    inputData[i+60]=str(pred[0])
    predlist.append(pred)

Streaming output truncated to the last 5000 lines.
0.6982546
1/1 [==============================] - 0s 10ms/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [==============================] - 0s 2ms/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [==============================] - 0s 974us/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [==============================] - 0s 1ms/step
0.6982546
1/1 [================

In [0]:
print(predlist)

# Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression 

## Model for past one hour values

In [ ]:
# for past one hour
regr = LinearRegression() 
regr.fit(X_train, y_train) 
prediction =regr.predict(X_test)

## Printing Errors

In [0]:
print('Training rmse:', np.sqrt(mean_squared_error(Y_train, regr.predict(X_train))))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_test, regr.predict(X_test))))
print('Validation Mean Absolute Error :',mean_absolute_error(Y_test, regr.predict(X_test)))

## Model for past  Two Hours

In [ ]:
def divide_chunks(l,step): 
  for i in range(0, len(l)-step):  
    yield l[i:i+step]
step=120
x = list(divide_chunks(dataset))
x=np.array(x)
x.shape=(2049220,60)
y_train=x[:,59:]
y_train=y_train[1:]
y_train=np.append(y_train,1.0)
l=len(y_train)
print(y_train.size)

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(x,labels,test_size=0.2)

In [ ]:
lin_reg1 = LinearRegression()
reg1 = lin_reg1.fit(X_train1, Y_train1)
prediction =reg1.predict(X_test1)
print("R2_score : ",r2_score(Y_test1,prediction))

In [ ]:
print('Training rmse:', np.sqrt(mean_squared_error(Y_train, reg1.predict(X_train1))))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_test, reg1.predict(X_test1))))
print('Validation Mean Absolute Error :',mean_absolute_error(Y_test, reg1.predict(X_test1)))

## Predicting Values based on past One hour

In [ ]:
# based on past one hour values
predlist=[]
for i in range(0,len(inputData)-60):
  if(inputData[i+60]=='?'):
    X_test=inputData[i:i+60]
    X_test=np.array(X_test)
    X_test= X_test.astype('float32')
    X_test.shape=(1,60)
    pred=regr.predict(X_test, verbose=1)
    pred.shape=(1,)
    print(pred[0])
    inputData[i+60]=str(pred[0])
    predlist.append(pred)

# Observation

| Model | window size | Accuracy | RMSE | MAE |
| --- | --- | --- | --- | --- |
| Linear Regression | 60 | 0.9397 | 0.2594 | 0.0978 |
| Linear Regression | 120 | 0.9398 | 0.2593 | 0.0978 |
| MLP | 60 | 0.9415 | 0.2165 | 0.0849 |
| MLP | 120 | 0.9408 | 0.2179 | 0.0905 |